In [1]:
import requests
import pandas as pd
import multiprocessing as mp
from functools import partial

url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
head = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36'}
payload = {
    'encodeURIComponent':'1',
    'step':'1',
    'firstin':'1',
    'off':'1',
    'TYPEK':'sii',
    'year':'',
    'season':''
}

In [2]:
def category(raw):
    
    series = raw[0:1]
    category1 = [] #儲存公司代號、公司名稱、營業收入、營業毛利、基本每股盈餘之index
    category2 = [] #儲存所有類別的index
    
    for i in series:
        category2.append(series[i].name)
        if series[i][0] == '公司代號' or series[i][0] == '公司名稱' or series[i][0] == '營業收入' or series[i][0] == '營業毛利（毛損）淨額' or series[i][0] == '基本每股盈餘（元）':
            category1.append(i)    
    
    category1 = set(category1) #轉成set
    category2 = set(category2) #轉成set
    category3 = list(category2 - category1) #category3為不要的類別
    
    table = raw.drop(category3, axis=1)#刪除不要的類別  
    table.columns = [0, 1, 2, 3, 4]#重設column index    
    
    return table

In [3]:
def multicore_season(year, pool):

    res = pool.map(season, year)#迭代相對應的次數
                                                        #partial目的：因pool只能傳入一個argument，故使用partial傳入multiple arguments
                                                        #http://python.omics.wiki/multiprocessing_map/multiprocessing_partial_function_multiple_arguments
    
    return res

In [6]:
def multicore_growth(year, pool):
    
    year = multicore_season(year, pool)  
    
    year1 = year[0]
    year2 = year[1]
    year3 = year[2]
    year4 = year[3]
    year5 = year[4]
    
    year = []
    
    for i in range(5):
        
    y.append([y_new, y_old])

    res = pool.map(growth, y)#迭代相對應的次數
                                                        #partial目的：因pool只能傳入一個argument，故使用partial傳入multiple arguments
                                                        #http://python.omics.wiki/multiprocessing_map/multiprocessing_partial_function_multiple_arguments
    
    return res

In [4]:
def season(year):
    
    payload['year'] = year[0]    
    payload['season'] = year[1]  
    
    req = requests.post(url, data = payload, headers = head)
    req.encoding = 'utf8'
    
    table = pd.read_html(req.text)[3] #第四張表格才是我們要的資料   
    table = category(table)#篩選出eps, 營收, 毛利
#     table = elec(table)#刪除非電子零件類股    
        
    return table

In [5]:
def growth(year):#y1新, y2舊

    #從year取出兩個年度
    year1 = year[0]
    year2 = year[1]
    
    #使y1, y2 公司名單相同
    y = compare_year(year1, year2)
    y1 = y[0]
    y2 = y[1]

    code = []#公司代號
    company = []#公司名稱
    income = []#營業收入
    profit = []#營業毛利（毛損）淨額
    eps = []#基本每股盈餘（元）

    for j in range(1, y1.shape[0]):
        code.append(y1[0][j])
        company.append(y1[1][j])
        income.append((float(y1[2][j]) - float(y2[2][j])) / float(y2[2][j]))#income成長率
        profit.append((float(y1[3][j]) - float(y2[3][j])) / float(y2[3][j]))#profit成長率

        #萬一eps為零，為了不讓分母為零，將其設為0.00000000000000000001
        if float(y2[4][j]) == 0:
            y2[4][j] = 0.00000000000000000001
        
        #分母必須為正，否則成長率有時會正負相反
        mother = float(y2[4][j])
        if mother < 0:
            mother = mother*(-1)
        eps.append((float(y1[4][j]) - float(y2[4][j])) / mother)#eps成長率

    s = {
        '公司代號':code,
        '公司名稱':company,
        '營業收入':income,
        '營業毛利（毛損）淨額':profit,
        '基本每股盈餘（元）':eps
    }
    s = pd.DataFrame(data = s)
        
    return s

#     return (year1, year2, y1, y2)

In [ ]:
def multi():
    res = pool.map(growth, y)

In [8]:
%%time
y1 = [106, 1]
y2 = [105, 1]
y3 = [104, 1]
y4 = [103, 1]
y5 = [102, 1]

y = [y1, y2, y3, y4, y5]



pool = mp.Pool()

g = multicore_growth(y, pool)

pool.close
pool.join



TypeError: 'int' object is not subscriptable

In [14]:
for i in range(4):
    locals()['abc'+str(i)+'de'] = i
    print(locals()['abc'+str(i)+'de'])
    


0
1
2
3


In [8]:
Y[0]

,0,1,2,3,4
0,公司代號,公司名稱,營業收入,營業毛利（毛損）淨額,基本每股盈餘（元）
1,1101,台泥,26716227,4864134,0.50
2,1102,亞泥,16256080,2237728,0.47
3,1103,嘉泥,1019429,58363,-0.06
4,1104,環球水泥,1379161,135602,0.40
5,1108,幸福水泥,1219347,166438,0.18
6,1109,信大水泥,1192024,259965,0.20
7,1110,東泥,495721,72028,0.12
8,1201,味全公司,6611781,2059871,0.51
9,1203,味王公司,1498954,379360,0.36
